# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.4/7729/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1007,Kevin,-0.24418217705854217,0.5798841947768838
2000-01-01 00:00:01,1040,Jerry,0.7718693591637251,-0.6035448434405932
2000-01-01 00:00:02,950,Yvonne,-0.26262145421267147,0.8333852442842391
2000-01-01 00:00:03,990,Michael,-0.041894396382261556,-0.28114205638658474
2000-01-01 00:00:04,1013,Ursula,-0.7493356078590572,-0.5136339191888442
2000-01-01 00:00:05,976,Xavier,0.5059517871245478,0.6760251081603263
2000-01-01 00:00:06,992,Oliver,-0.9848444438829169,0.5154792403894435
2000-01-01 00:00:07,1051,Patricia,-0.8370438269358174,0.7525684341603387
2000-01-01 00:00:08,1000,Jerry,-0.870979789006024,-0.26191095730087155


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,975,Quinn,0.48627835388117413,0.14203527147398942
2000-01-30 00:00:01,1005,Sarah,-0.19868563688585317,0.9402561667409959
2000-01-30 00:00:02,992,Dan,-0.941175909246329,-0.46189135654914737
2000-01-30 00:00:03,989,Ingrid,0.3765413894964491,-0.8718087969431174
2000-01-30 00:00:04,998,Xavier,-0.26449427593221997,0.7724172292589739
2000-01-30 00:00:05,981,Norbert,0.08407843815586413,-0.9269915923066832
2000-01-30 00:00:06,983,Yvonne,-0.4257934330983133,-0.3108432939487511
2000-01-30 00:00:07,987,George,0.17457825796090298,-0.3180056448267743
2000-01-30 00:00:08,998,Edith,-0.12477632638601532,-0.7863269242749089


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1007,Kevin,-0.244182,0.579884
1,2000-01-01 00:00:01,1040,Jerry,0.771869,-0.603545
2,2000-01-01 00:00:02,950,Yvonne,-0.262621,0.833385
3,2000-01-01 00:00:03,990,Michael,-0.041894,-0.281142
4,2000-01-01 00:00:04,1013,Ursula,-0.749336,-0.513634


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1007,Kevin,-0.244182,0.579884
1,2000-01-01 00:00:01,1040,Jerry,0.771869,-0.603545
2,2000-01-01 00:00:02,950,Yvonne,-0.262621,0.833385
3,2000-01-01 00:00:03,990,Michael,-0.041894,-0.281142
4,2000-01-01 00:00:04,1013,Ursula,-0.749336,-0.513634


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.59 s, sys: 441 ms, total: 4.03 s
Wall time: 3.32 s


name
Alice      -0.000439
Bob        -0.000207
Charlie     0.001196
Dan        -0.000797
Edith      -0.001696
Frank      -0.001432
George     -0.002640
Hannah     -0.000786
Ingrid      0.002652
Jerry       0.003414
Kevin       0.001917
Laura      -0.003167
Michael    -0.001267
Norbert     0.000316
Oliver     -0.000947
Patricia    0.001523
Quinn      -0.001202
Ray        -0.001073
Sarah       0.001772
Tim         0.001127
Ursula      0.001891
Victor     -0.001742
Wendy      -0.001471
Xavier     -0.000035
Yvonne      0.001535
Zelda      -0.002146
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.02 s, sys: 142 ms, total: 1.16 s
Wall time: 1.04 s


name
Alice      -0.000439
Bob        -0.000207
Charlie     0.001196
Dan        -0.000797
Edith      -0.001696
Frank      -0.001432
George     -0.002640
Hannah     -0.000786
Ingrid      0.002652
Jerry       0.003414
Kevin       0.001917
Laura      -0.003167
Michael    -0.001267
Norbert     0.000316
Oliver     -0.000947
Patricia    0.001523
Quinn      -0.001202
Ray        -0.001073
Sarah       0.001772
Tim         0.001127
Ursula      0.001891
Victor     -0.001742
Wendy      -0.001471
Xavier     -0.000035
Yvonne      0.001535
Zelda      -0.002146
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.05 s, sys: 48.5 ms, total: 1.1 s
Wall time: 978 ms


name
Alice      -0.000439
Bob        -0.000207
Charlie     0.001196
Dan        -0.000797
Edith      -0.001696
Frank      -0.001432
George     -0.002640
Hannah     -0.000786
Ingrid      0.002652
Jerry       0.003414
Kevin       0.001917
Laura      -0.003167
Michael    -0.001267
Norbert     0.000316
Oliver     -0.000947
Patricia    0.001523
Quinn      -0.001202
Ray        -0.001073
Sarah       0.001772
Tim         0.001127
Ursula      0.001891
Victor     -0.001742
Wendy      -0.001471
Xavier     -0.000035
Yvonne      0.001535
Zelda      -0.002146
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html